In [ ]:
import numpy as np
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModel
import torch
import faiss
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, models
import torch.nn.functional as F


# Data Load

In [2]:
df = pd.read_csv('../quran_id_full.csv')
df = df[['id', 'suraId', 'verseID', 'indoText']]

In [20]:
query = "apa itu jalan yang lurus"

In [7]:
corpus = df.indoText.tolist()

# Model Init

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [44]:
model1_name = 'sentence-transformers/msmarco-distilbert-base-dot-prod-v3'
model2_name = 'denaya/indoSBERT-large'

In [6]:
models = [model1_name, model2_name]

# SBERT Method Compare

difference between sbert embedding with classic method

## SBERT embedding

In [8]:
sbert_model = SentenceTransformer(model1_name, device=device)
encoded_data = sbert_model.encode(corpus, convert_to_tensor=True)
# encoded_data = np.asarray(encoded_data.astype('float32'))

In [21]:
encoded_query = sbert_model.encode(query, convert_to_tensor=True)
similarity_scores = sbert_model.similarity(encoded_query, encoded_data)[0]
scores, indices = torch.topk(similarity_scores, k=10)

In [22]:
for score, idx in zip(scores, indices):
    print(f"Ayat: {corpus[idx]}\nScore: {score}\n")

Ayat: (yang berada) di atas jalan yang lurus,
Score: 55.79570007324219

Ayat: Dan hak Allah menerangkan jalan yang lurus, dan di antaranya ada (jalan) yang menyimpang. Dan jika Dia menghendaki, tentu Dia memberi petunjuk kamu semua (ke jalan yang benar).
Score: 53.78508758544922

Ayat: (yaitu) bagi siapa di antara kamu yang menghendaki menempuh jalan yang lurus.
Score: 52.73957061767578

Ayat: (Isa berkata), “Dan sesungguhnya Allah itu Tuhanku dan Tuhanmu, maka sembahlah Dia. Ini adalah jalan yang lurus.”
Score: 52.0804328918457

Ayat: Dan di antara kami ada yang Islam dan ada yang menyimpang dari kebenaran. Siapa yang Islam, maka mereka itu telah memilih jalan yang lurus.
Score: 51.20023727416992

Ayat: Sesungguhnya orang-orang yang kafir itu, menginfakkan harta mereka untuk menghalang-halangi (orang) dari jalan Allah. Mereka akan (terus) menginfakkan harta itu, kemudian mereka akan menyesal sendiri, dan akhirnya mereka akan dikalahkan. Ke dalam neraka Jahanamlah orang-orang kafir itu

## Denaya

In [47]:
sbert_model2 = SentenceTransformer(model2_name, device=device)
encoded_data = sbert_model2.encode(corpus, convert_to_tensor=True)

model.safetensors:   5%|4         | 62.9M/1.34G [00:00<?, ?B/s]

In [49]:
encoded_query = sbert_model2.encode(query, convert_to_tensor=True)
similarity_scores = sbert_model2.similarity(encoded_query, encoded_data)[0]
scores, indices = torch.topk(similarity_scores, k=10)

In [50]:
for score, idx in zip(scores, indices):
    print(f"Ayat: {corpus[idx]}\nScore: {score}\n")

Ayat: (yang berada) di atas jalan yang lurus,
Score: 0.8601018190383911

Ayat: Dia (Allah) berfirman, “Ini adalah jalan yang lurus (menuju) kepada-Ku.”
Score: 0.7590444087982178

Ayat: Tunjukilah kami jalan yang lurus,
Score: 0.7579336166381836

Ayat: (yaitu) bagi siapa di antara kamu yang menghendaki menempuh jalan yang lurus.
Score: 0.7277039289474487

Ayat: Demi langit yang mempunyai jalan-jalan,
Score: 0.7072363495826721

Ayat: dan pasti Kami tunjukkan kepada mereka jalan yang lurus.
Score: 0.6908457279205322

Ayat: Dan hak Allah menerangkan jalan yang lurus, dan di antaranya ada (jalan) yang menyimpang. Dan jika Dia menghendaki, tentu Dia memberi petunjuk kamu semua (ke jalan yang benar).
Score: 0.6743772625923157

Ayat: Dan sesungguhnya engkau pasti telah menyeru mereka kepada jalan yang lurus.
Score: 0.667270302772522

Ayat: di dalamnya terdapat (isi) kitab-kitab yang lurus (benar).
Score: 0.66031813621521

Ayat: Maka berpegang teguhlah engkau kepada (agama) yang telah diwahyuka

## AutoTokenizer 

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model1_name)
model = AutoModel.from_pretrained(model1_name)

In [12]:
def get_embedding(text, tokenizer, model):
    # Tokenizing the input text
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    
    # Print the tokenized components
    # tokens = tokenizer.tokenize(text)  # Get tokens as text
    # token_ids = inputs['input_ids'][0].numpy()  # Convert tensor to numpy array for readability
    
    # # Display tokenization details
    # print(f"Original Text: {text}")
    # print(f"Tokens: {tokens}")
    # print(f"Token IDs: {token_ids}")
    # print(f"Attention Mask: {inputs['attention_mask'][0].numpy()}")
    # print("\n" + "="*50 + "\n")
    
    # Pass through model without computing gradients
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Return the embedding from [CLS] token representation
    return outputs.last_hidden_state[:, 0, :].squeeze(0) 

In [31]:
ayat_embeddings_sbert = [get_embedding(ayat, tokenizer, model) for ayat in corpus]

In [30]:
query_embedding_sbert = get_embedding(query, tokenizer, model)

In [32]:
# Calculate cosine similarity
# similarities = [cosine_similarity(query_embedding, ayat_embedding)[0][0] for ayat_embedding in ayat_embeddings]
similarities = [torch.dot(query_embedding_sbert, ayat_embedding).item() for ayat_embedding in ayat_embeddings_sbert]

# Rank ayat by similarity
ranking = sorted(zip(corpus, similarities), key=lambda x: x[1], reverse=True)

# Print ranked ayat
for ayat, score in ranking:
    print(f"Ayat: {ayat}\nScore: {score}\n")

Ayat: Bukankah Kami telah menjadikan untuknya sepasang mata,
Score: 428.4288024902344

Ayat: kemudian Dia mematikannya lalu menguburkannya,
Score: 428.1026916503906

Ayat: sedang dia takut (kepada Allah),
Score: 424.635986328125

Ayat: Dan gunung-gunung bagaimana ditegakkan?
Score: 419.95794677734375

Ayat: Kamukah yang menciptakannya, ataukah Kami penciptanya?
Score: 414.4361877441406

Ayat: Apabila bumi diguncangkan sedahsyat-dahsyatnya,
Score: 406.6830139160156

Ayat: kemudian Kami belah bumi dengan sebaik-baiknya,
Score: 406.1723327636719

Ayat: Lalu para malaikat itu bersujud semuanya,
Score: 402.7583312988281

Ayat: dan langit, bagaimana ditinggikan?
Score: 400.5941162109375

Ayat: mereka itu memperoleh rezeki yang sudah ditentukan,
Score: 400.21728515625

Ayat: Mereka tinggal di sana dalam masa yang lama,
Score: 398.1305847167969

Ayat: Di sana ada mata air yang mengalir.
Score: 398.0472412109375

Ayat: Bahkan manusia menjadi saksi atas dirinya sendiri,
Score: 396.1717834472656


# SBERT vs indobert

## Indobert

In [33]:
tokenizer = AutoTokenizer.from_pretrained(model2_name)
model = AutoModel.from_pretrained(model2_name)

In [ ]:
ayat_embeddings_indobert = [get_embedding(ayat, tokenizer, model) for ayat in corpus]
ayat_embeddings_indobert = [ayat.reshape(1, -1) for ayat in ayat_embeddings_indobert]

In [38]:
query_embedding_indobert = get_embedding(query, tokenizer, model)
query_embedding_indobert.reshape(1, -1)

tensor([[ 1.2724e+00, -1.4689e+00, -5.6605e-01, -5.6759e-02,  5.1447e-01,
          5.4387e-01, -1.3386e+00, -6.6781e-02,  9.3850e-01, -8.2752e-02,
         -1.0156e+00, -3.2174e-01,  4.1641e-01,  2.2798e-01, -9.6003e-01,
         -5.2328e-01, -5.8380e-01,  1.0278e+00,  1.0091e+00, -1.4218e+00,
          7.0414e-01, -5.1171e-03,  8.1370e-01,  2.6872e-02, -1.4076e+00,
          1.6010e-02,  7.8466e-01,  6.7906e-01, -3.6127e-01,  1.3430e+00,
         -6.9057e-01, -8.1474e-01,  2.6058e-01, -5.7250e-01,  7.7683e-01,
         -4.5440e-01,  3.7197e-01, -1.5467e+00, -6.2999e-01,  1.9202e-01,
          7.5064e-02,  4.4564e-01,  5.7052e-01,  6.6837e-01,  7.7885e-01,
          1.9194e-01,  1.2612e-01, -3.4817e-02,  7.8642e-01, -2.0393e-01,
         -1.2191e-01, -1.0682e+00, -1.6077e+00,  7.5807e-01, -7.1166e-01,
         -2.7974e-01,  1.5111e+00, -2.3163e-01, -7.0402e-01,  1.0442e+00,
          1.4910e+00, -1.0404e+00, -6.6112e-01, -2.1417e+00,  8.0576e-01,
         -2.0699e-01, -4.6933e-01,  1.

In [40]:
similarities_indobert = [cosine_similarity(query_embedding_indobert, ayat_embedding)[0][0] for ayat_embedding in ayat_embeddings_indobert]

ValueError: Expected 2D array, got 1D array instead:
array=[ 1.27240944e+00 -1.46888101e+00 -5.66045165e-01 -5.67594469e-02
  5.14466405e-01  5.43865681e-01 -1.33861673e+00 -6.67814761e-02
  9.38498914e-01 -8.27517435e-02 -1.01555431e+00 -3.21737260e-01
  4.16410863e-01  2.27979332e-01 -9.60025609e-01 -5.23281515e-01
 -5.83797455e-01  1.02775407e+00  1.00913143e+00 -1.42180252e+00
  7.04140007e-01 -5.11714816e-03  8.13704014e-01  2.68715322e-02
 -1.40764546e+00  1.60099491e-02  7.84663737e-01  6.79059267e-01
 -3.61270428e-01  1.34299457e+00 -6.90567672e-01 -8.14739585e-01
  2.60584146e-01 -5.72502375e-01  7.76827812e-01 -4.54397351e-01
  3.71966541e-01 -1.54666591e+00 -6.29993677e-01  1.92015395e-01
  7.50638172e-02  4.45643067e-01  5.70520163e-01  6.68367565e-01
  7.78851569e-01  1.91935897e-01  1.26116097e-01 -3.48169804e-02
  7.86417365e-01 -2.03933969e-01 -1.21911287e-01 -1.06816077e+00
 -1.60765433e+00  7.58074045e-01 -7.11655080e-01 -2.79739499e-01
  1.51109600e+00 -2.31631815e-0

In [ ]:
# Rank ayat by similarity
ranking = sorted(zip(corpus, similarities_indobert), key=lambda x: x[1], reverse=True)

# Print ranked ayat
for ayat, score in ranking:
    print(f"Ayat: {ayat}\nScore: {score}\n")